### **Initialisation:**

In [0]:
# Clone repo
!git clone https://github.com/MatchLab-Imperial/keras_triplet_descriptor

Cloning into 'keras_triplet_descriptor'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 181 (delta 0), reused 1 (delta 0), pack-reused 178
Receiving objects: 100% (181/181), 149.87 MiB | 8.43 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Checking out files: 100% (69/69), done.


In [0]:
# Change directory
%cd /content/keras_triplet_descriptor  

/content/keras_triplet_descriptor


In [0]:
# Download data
!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip


--2019-03-22 19:22:43--  https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Resolving imperialcollegelondon.box.com (imperialcollegelondon.box.com)... 107.152.25.197
Connecting to imperialcollegelondon.box.com (imperialcollegelondon.box.com)|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
--2019-03-22 19:22:43--  https://imperialcollegelondon.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Reusing existing connection to imperialcollegelondon.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://imperialcollegelondon.app.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
--2019-03-22 19:22:43--  https://imperialcollegelondon.app.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Resolving imperialcollegelondon.app.box.com (imperialcollegelondon.app.box.

KeyboardInterrupt: ignored

In [0]:
# Extract data
!unzip -q ./hpatches_data.zip
!rm ./hpatches_data.zip

In [0]:
import sys
import json
import os
import glob
import time
import tensorflow as tf
import numpy as np
import cv2
import random
import os, glob, datetime
import keras
import argparse
import re

from keras import backend as K
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization 
from keras.layers import Input, UpSampling2D, concatenate, Activation,Subtract
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K

from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet

In [0]:
#initialise seed
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

In [0]:
#define directories
hpatches_dir = './hpatches'
splits_path = './splits.json'


splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']

train_fnames = split['train']
test_fnames = split['test']

seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('/')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('/')[-1] in split['test'], seqs)) 


In [0]:
def get_denoise_model_baseline(shape):
    
  inputs = Input(shape)
  
  ## Encoder starts
  conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
  ## Bottleneck
  conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)

  ## Now the decoder starts
  up3 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv2))
  merge3 = concatenate([conv1,up3], axis = -1)
  conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge3)
    
  conv4 = Conv2D(1, 3,  padding = 'same')(conv3)

  shallow_net = Model(inputs = inputs, outputs = conv4)
  
  return shallow_net

def get_denoise_model_UNET(shape):
    
  inputs = Input(shape)
  
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  drop5 = Dropout(0.5)(conv5)

  ## Now the decoder starts

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv10 = Conv2D(1, 3,  padding = 'same')(conv9)

  UNET2015 = Model(inputs = inputs, outputs = conv10)
  
  return UNET2015



def get_descriptor_model(shape):
  
  '''Architecture copies HardNet architecture'''
  
  init_weights = keras.initializers.he_normal()
  
  descriptor_model = Sequential()
  descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(Dropout(0.3))

  descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init_weights))
  
  # Final descriptor reshape
  descriptor_model.add(Reshape((128,)))
  
  return descriptor_model
  
  
def triplet_loss(x):
  
  output_dim = 128
  a, p, n = x
  _alpha = 1.0
  positive_distance = K.mean(K.square(a - p), axis=-1)
  negative_distance = K.mean(K.square(a - n), axis=-1)
  
  return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)


def DnCNN(depth,filters=64,image_channels=1, use_bnorm=True):
    layer_count = 0
    inpt = Input(shape=(None,None,image_channels),name = 'input'+str(layer_count))
    # 1st layer, Conv+relu
    layer_count += 1
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',name = 'conv'+str(layer_count))(inpt)
    layer_count += 1
    x = Activation('relu',name = 'relu'+str(layer_count))(x)
    # depth-2 layers, Conv+BN+relu
    for i in range(depth-2):
        layer_count += 1
        x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
        if use_bnorm:
            layer_count += 1
            #x = BatchNormalization(axis=3, momentum=0.1,epsilon=0.0001, name = 'bn'+str(layer_count))(x
        x = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(layer_count))(x)
        layer_count += 1
        x = Activation('relu',name = 'relu'+str(layer_count))(x)  
    # last layer, Conv
    layer_count += 1
    x = Conv2D(filters=image_channels, kernel_size=(3,3), strides=(1,1), kernel_initializer='Orthogonal',padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
    layer_count += 1
    x = Subtract(name = 'subtract' + str(layer_count))([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    
    return model
  
  # define loss
def sum_squared_error(y_true, y_pred):
    #return K.mean(K.square(y_pred - y_true), axis=-1)
    #return K.sum(K.square(y_pred - y_true), axis=-1)/2
    return K.sum(K.square(y_pred - y_true))/2
  
def lr_schedule(epoch):
    initial_lr = 0.001
    if epoch<=30:
        lr = initial_lr
    elif epoch<=60:
        lr = initial_lr/10
    elif epoch<=80:
        lr = initial_lr/20 
    else:
        lr = initial_lr/20 
    return lr

In [0]:
#Load Subset of NHPatches for Denoising
denoise_generator = DenoiseHPatches(random.sample(seqs_train, 3), batch_size=50)
denoise_generator_val = DenoiseHPatches(random.sample(seqs_test, 1), batch_size=50)

NameError: ignored

### **EVALUATION OF DIFFERENT DENOISE MODELS:**

In [0]:
#Load Shallow UNET for Base Line Evaluation
shape = (32, 32, 1)
denoise_model = get_denoise_model_baseline(shape)


In [0]:
#Train the baseline UNET for 30 epochs in subset
epochs = 30
denoise_history_full=[]
sgd = keras.optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)
denoise_model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mae'])
### Use a loop to save for each epoch the weights in an external website in
### case colab stops. Every time you call fit/fit_generator the weigths are NOT
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)
for e in range(epochs):
  denoise_history = denoise_model.fit_generator(generator=denoise_generator, 
                                                epochs=1, verbose=1, 
                                                validation_data=denoise_generator_val, callbacks=[lr_scheduler])
  denoise_history_full.append(denoise_history)
  ### Saves optimizer and weights
  denoise_model.save('denoise_baseline.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@denoise_baseline.h5" https://file.io
  print('Epoch Number= {}'.format(e))

In [0]:
for epoch in denoise_history_full:
  print(epoch.history)

In [0]:
plot_denoise(denoise_model)

In [0]:
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
#Must import 'descriptorbaselinecleansmall.h5' into directory
descriptor_model_trip.set_weights(keras.models.load_model('./descriptorbaselinecleansmall.h5').get_weights())
descriptor_model_trip.optimizer = keras.models.load_model('./descriptorbaselinecleansmall.h5').optimizer

In [0]:
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)

In [0]:
#Patch Verification:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

In [0]:
#Image Matching
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

In [0]:
#Patch Retrieval
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

In [0]:
#Load Full UNET for Evaluation
shape = (32, 32, 1)
denoise_model = get_denoise_model_UNET(shape)


In [0]:
#Train the  UNET for 30 epochs in subset
epochs = 30
sgd = keras.optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)
denoise_model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mae'])
denoise_history_full=[]
### Use a loop to save for each epoch the weights in an external website in
### case colab stops. Every time you call fit/fit_generator the weigths are NOT
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)
for e in range(epochs):
  denoise_history = denoise_model.fit_generator(generator=denoise_generator, 
                                                epochs=1, verbose=1, 
                                                validation_data=denoise_generator_val, callbacks=[lr_scheduler])
  denoise_history_full.append(denoise_history)
  ### Saves optimizer and weights
  denoise_model.save('denoise_UNET.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@denoise_UNET.h5" https://file.io
  print('Epoch Number= {}'.format(e))

In [0]:
for epoch in denoise_history_full:
  print(epoch.history)

In [0]:
plot_denoise(denoise_model)

In [0]:
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
#Must import 'descriptorbaselinecleansmall.h5' into directory
descriptor_model_trip.set_weights(keras.models.load_model('./descriptorbaselinecleansmall.h5').get_weights())
descriptor_model_trip.optimizer = keras.models.load_model('./descriptorbaselinecleansmall.h5').optimizer

In [0]:
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)

In [0]:
#Patch Verification:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

In [0]:
#Image Matching
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

In [0]:
#Patch Retrieval
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

In [0]:
#Must upload model.h5
#Load DnCNN +Transfer Network
shape = (32, 32, 1)

model_Dnn=DnCNN(depth=17,filters=64,image_channels=1,use_bnorm=True)
model_Dnn.compile(optimizer=Adam(0.001), loss=sum_squared_error)
model_Dnn.load_weights('./model.h5')
model_Dnn.summary()
#define my model for end of denoiser

#First Layers
inpt = Input(shape=(None,None,1),name = 'input'+str(1))
conv1 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(1))(inpt)
activation1=Activation('relu',name = 'relu'+str(1))(conv1)
#Middle Layers
conv2 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(2))(activation1)
Norm2 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(2))(conv2)
activation2=Activation('relu',name = 'relu'+str(2))(Norm2)

conv3 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(3))(activation2)
Norm3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(3))(conv3)
activation3=Activation('relu',name = 'relu'+str(3))(Norm3)

conv4 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(4))(activation3)
Norm4 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(4))(conv4)
activation4=Activation('relu',name = 'relu'+str(4))(Norm4)
#Final Layers                       
conv5 = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(5))(activation4)
Norm5 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(5))(conv5)
ErrorLayer=Subtract(name = 'subtract' + str(5))([inpt, Norm5])

my_model=Model(inputs=inpt, outputs=ErrorLayer)
my_model.summary()
denoise_model=Sequential()
denoise_model.add(model_Dnn)
denoise_model.add(my_model)

for layer in model_Dnn.layers:
	layer.trainable = False
  
denoise_model.summary()

for layer in model_Dnn.layers:
	layer.trainable = False
  
sgd = keras.optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)
denoise_model.compile(optimizer=Adam(0.001), loss=sum_squared_error)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 640         input0[0][0]                     
__________________________________________________________________________________________________
relu2 (Activation)              (None, None, None, 6 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, None, None, 6 36864       relu2[0][0]                      
__________________________________________________________________________________________________
bn4 (Batch

In [0]:
#Train the DnCNN for 30 epochs in subset
epochs = 30
#denoise_model= keras.models.load_model('./Final denoise 2 epoch.h5',custom_objects={'sum_squared_error':sum_squared_error})
lr_scheduler = LearningRateScheduler(lr_schedule)
denoise_history_full=[]
### Use a loop to save for each epoch the weights in an external website in
### case colab stops. Every time you call fit/fit_generator the weigths are NOT
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)
for e in range(epochs):
  denoise_history = denoise_model.fit_generator(generator=denoise_generator, 
                                                epochs=1, verbose=1, 
                                                validation_data=denoise_generator_val, callbacks=[lr_scheduler])
  denoise_history_full.append(denoise_history)
  ### Saves optimizer and weights
  denoise_model.save('denoise_DnCNN.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@denoise_DnCNN.h5" https://file.io
  print('Epoch Number= {}'.format(e))

In [0]:
for epoch in denoise_history_full:
  print(epoch.history)

In [0]:
plot_denoise(denoise_model)

In [0]:
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
#Must import 'descriptorbaselinecleansmall.h5' into directory
descriptor_model_trip.set_weights(keras.models.load_model('./descriptorbaselinecleansmall.h5').get_weights())
descriptor_model_trip.optimizer = keras.models.load_model('./descriptorbaselinecleansmall.h5').optimizer

In [0]:
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)

In [0]:
#Patch Verification:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

In [0]:
#Image Matching
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

In [0]:
#Patch Retrieval
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

### **DESCRIPTOR NOISY DENOISED CLEAN TESTS**

In [0]:
#Training Baseline Descriptor on Noisy
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=sgd)

In [0]:
### Descriptor loading and training for 10,0000 Triplets
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=False)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=10000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=1000)

In [0]:
plot_triplet(training_generator)

In [0]:
epochs = 50
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer
descriptor_history_full=[]
for e in range(epochs):
  
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  descriptor_history_full.append(descriptor_history)
  ### Saves optimizer and weights
  descriptor_model_trip.save('descriptor_noisy.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@descriptor_noisy.h5" https://file.io
  print("EPOCH NUMBER: {}".format(e))

In [0]:
for epoch in descriptor_history_full:
  print(epoch.history)

In [0]:
#Must Include 'UNETsmall.h5' in directory
shape = (32, 32, 1)
denoise_model=get_denoise_model_UNET(shape)
denoise_model= keras.models.load_model('./UNETsmall.h5')
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)

In [0]:
#Patch Verification:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

In [0]:
#Image Matching
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

In [0]:
#Patch Retrieval
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

In [0]:
#Training Baseline Descriptor on Clean
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=sgd)

In [0]:
### Descriptor loading and training for 10,0000 Triplets
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=True)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=10000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=1000)

In [0]:
plot_triplet(training_generator)

In [0]:
epochs = 50
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer
descriptor_history_full=[]
for e in range(epochs):
  
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  descriptor_history_full.append(descriptor_history)
  ### Saves optimizer and weights
  descriptor_model_trip.save('descriptor_clean.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@descriptor_clean.h5" https://file.io
  print("EPOCH NUMBER: {}".format(e))

In [0]:
for epoch in descriptor_history_full:
  print(epoch.history)

In [0]:
#Must Include 'UNETsmall.h5' in directory
shape = (32, 32, 1)
denoise_model=get_denoise_model_UNET(shape)
denoise_model= keras.models.load_model('./UNETsmall.h5')
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)

In [0]:
#Patch Verification:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

In [0]:
#Image Matching
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

In [0]:
#Patch Retrieval
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

In [0]:
#Training Baseline Descriptor on Denoised
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=sgd)

In [0]:
#Must Include 'UNETsmall.h5' in directory
shape = (32, 32, 1)
denoise_model=get_denoise_model_UNET(shape)
denoise_model= keras.models.load_model('./UNETsmall.h5')

### Descriptor loading and training for 10,0000 Triplets
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=False)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=10000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=1000)

In [0]:
plot_triplet(training_generator)

In [0]:
epochs = 50
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer
descriptor_history_full=[]
for e in range(epochs):
  
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  descriptor_history_full.append(descriptor_history)
  ### Saves optimizer and weights
  descriptor_model_trip.save('descriptor_denoised.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@descriptor_denoised.h5" https://file.io
  print("EPOCH NUMBER: {}".format(e))

In [0]:
for epoch in descriptor_history_full:
  print(epoch.history)

In [0]:
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=True)

In [0]:
#Patch Verification:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

In [0]:
#Image Matching
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

In [0]:
#Patch Retrieval
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

### **TRIPLET GENERATION Tests:**

In [0]:
#Testing Tough Positv
#Must include 'read_data_tough_postive.py'
from read_data_tough_positive import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps

In [0]:
#Training Baseline Descriptor for tough_positives on clean images:

from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=sgd)

In [0]:
### Descriptor loading and training for 10,0000 Triplets Trained on Clean 
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=True)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=10000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=1000)

In [0]:
plot_triplet(training_generator)

In [0]:
epochs = 50
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer
descriptor_history_full=[]
for e in range(epochs):
  
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  descriptor_history_full.append(descriptor_history)
  ### Saves optimizer and weights
  descriptor_model_trip.save('descriptor_toughpositive.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@descriptor_toughpositive.h5" https://file.io
  print("EPOCH NUMBER: {}".format(e))

In [0]:
for epoch in descriptor_history_full:
  print(epoch.history)

In [0]:
#tested on clean images upperbound
generate_desc_csv(descriptor_model, seqs_test, denoise_model=None, use_clean=True)

In [0]:
#Patch Verification:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

In [0]:
#Image Matching
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

In [0]:
#Patch Retrieval
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

In [0]:
#Testing farnegative
#Must include 'read_data_farnegative.py'
from read_data_tough_farnegative import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps

In [0]:
#Training Baseline Descriptor for tough_positives on clean images:

from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=sgd)

In [0]:
### Descriptor loading and training for 10,0000 Triplets Trained on Clean 
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=True)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=10000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=1000)

In [0]:
plot_triplet(training_generator)

In [0]:
epochs = 50
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer
descriptor_history_full=[]
for e in range(epochs):
  
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  descriptor_history_full.append(descriptor_history)
  ### Saves optimizer and weights
  descriptor_model_trip.save('descriptor_farnegative.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@descriptor_farnegative.h5" https://file.io
  print("EPOCH NUMBER: {}".format(e))

In [0]:
for epoch in descriptor_history_full:
  print(epoch.history)

In [0]:
#tested on clean images upperbound
generate_desc_csv(descriptor_model, seqs_test, denoise_model=None, use_clean=True)

In [0]:
#Patch Verification:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

In [0]:
#Image Matching
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

In [0]:
#Patch Retrieval
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

### **FINAL PROPOSED APPROACH:**

In [0]:
from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps

In [0]:
#initialise seed
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

In [0]:

#Load Full Data Set
denoise_generator = DenoiseHPatches(seqs_train, batch_size=50)
denoise_generator_val = DenoiseHPatches(seqs_test, batch_size=50)

100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


In [0]:
#Load DnCNN +Transfer Network
shape = (32, 32, 1)

model_Dnn=DnCNN(depth=17,filters=64,image_channels=1,use_bnorm=True)
model_Dnn.compile(optimizer=Adam(0.001), loss=sum_squared_error)
model_Dnn.load_weights('./model.h5')
model_Dnn.summary()
#define my model for end of denoiser

#First Layers
inpt = Input(shape=(None,None,1),name = 'input'+str(1))
conv1 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(1))(inpt)
activation1=Activation('relu',name = 'relu'+str(1))(conv1)
#Middle Layers
conv2 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(2))(activation1)
Norm2 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(2))(conv2)
activation2=Activation('relu',name = 'relu'+str(2))(Norm2)

conv3 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(3))(activation2)
Norm3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(3))(conv3)
activation3=Activation('relu',name = 'relu'+str(3))(Norm3)

conv4 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(4))(activation3)
Norm4 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(4))(conv4)
activation4=Activation('relu',name = 'relu'+str(4))(Norm4)
#Final Layers                       
conv5 = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(5))(activation4)
Norm5 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(5))(conv5)
ErrorLayer=Subtract(name = 'subtract' + str(5))([inpt, Norm5])

my_model=Model(inputs=inpt, outputs=ErrorLayer)
my_model.summary()
denoise_model=Sequential()
denoise_model.add(model_Dnn)
denoise_model.add(my_model)

for layer in model_Dnn.layers:
	layer.trainable = False
  
denoise_model.summary()

for layer in model_Dnn.layers:
	layer.trainable = False
  
sgd = keras.optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)
denoise_model.compile(optimizer=Adam(0.001), loss=sum_squared_error)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 640         input0[0][0]                     
__________________________________________________________________________________________________
relu2 (Activation)              (None, None, None, 6 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, None, None, 6 36864       relu2[0][0]                      
__________________________________________________________________________________________________
bn4 (Batch

In [0]:
#Train the DnCNN for 30 epochs on Full 
epochs = 30
#denoise_model= keras.models.load_model('./Final denoise 2 epoch.h5',custom_objects={'sum_squared_error':sum_squared_error})
lr_scheduler = LearningRateScheduler(lr_schedule)
denoise_history_full=[]
### Use a loop to save for each epoch the weights in an external website in
### case colab stops. Every time you call fit/fit_generator the weigths are NOT
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)
for e in range(epochs):
  denoise_history = denoise_model.fit_generator(generator=denoise_generator, 
                                                epochs=1, verbose=1, 
                                                validation_data=denoise_generator_val, callbacks=[lr_scheduler])
  denoise_history_full.append(denoise_history)
  ### Saves optimizer and weights
  denoise_model.save('denoise_DnCNN_full.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@denoise_DnCNN_full.h5" https://file.io
  print('Epoch Number= {}'.format(e))

Epoch 1/1
 123/1545 [=>............................] - ETA: 1:19:10 - loss: 12090194.2683

KeyboardInterrupt: ignored

In [0]:
from read_data_tough_positive import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps

In [0]:
#Load Baseline Descriptor
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
sgd = keras.optimizers.SGD(lr=0.1)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=sgd)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
### Descriptor loading and training for 10,0000 Triplets Trained on Clean using Tough Positive 
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=True)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=100000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=10000)

Using clean patches
100%|██████████| 116/116 [00:35<00:00,  3.30it/s]


100%|██████████| 10000/10000 [00:00<00:00, 23089.60it/s]

Using clean patches
  0%|          | 0/116 [00:00<?, ?it/s]

100%|██████████| 116/116 [00:22<00:00,  3.11it/s]


100%|██████████| 1000/1000 [00:00<00:00, 64449.42it/s]


In [0]:
#Train on tough_positive for 10 epochs
epochs = 10
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer
descriptor_history_full=[]
for e in range(epochs):
  
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  descriptor_history_full.append(descriptor_history)
  ### Saves optimizer and weights
  descriptor_model_trip.save('descriptor_farnegative.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@descriptor_farnegative.h5" https://file.io
  print("EPOCH NUMBER: {}".format(e))

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
199/200 [============================>.] - ETA: 2s - loss: 0.3514

100%|██████████| 10000/10000 [00:00<00:00, 33598.24it/s]


200/200 [==============================] - 473s 2s/step - loss: 0.3508 - val_loss: 0.5961


100%|██████████| 1000/1000 [00:00<00:00, 60996.52it/s]


{"success":true,"key":"t7VmyB","link":"https://file.io/t7VmyB","expiry":"14 days"}EPOCH NUMBER: 0
Epoch 1/1
199/200 [============================>.] - ETA: 2s - loss: 0.2909

...

{"success":true,"key":"6PcWm8","link":"https://file.io/6PcWm8","expiry":"14 days"}EPOCH NUMBER: 8
Epoch 1/1
199/200 [============================>.] - ETA: 2s - loss: 0.3913

100%|██████████| 10000/10000 [00:00<00:00, 22972.19it/s]


200/200 [==============================] - 476s 2s/step - loss: 0.3926 - val_loss: 0.4289


100%|██████████| 1000/1000 [00:00<00:00, 68917.25it/s]


{"success":true,"key":"L6rMMh","link":"https://file.io/L6rMMh","expiry":"14 days"}EPOCH NUMBER: 9


In [0]:
for epoch in descriptor_history_full:
  print(epoch.history)

{'val_loss': [0.5960797265172004], 'loss': [0.35076710890978574]}
{'val_loss': [0.4094816155731678], 'loss': [0.2912413223832846]}
{'val_loss': [0.36746436953544614], 'loss': [0.2584550421498716]}
{'val_loss': [0.32817778959870336], 'loss': [0.27568794777151195]}
{'val_loss': [0.3477799784392118], 'loss': [0.27408573728054764]}
{'val_loss': [0.3259496470913291], 'loss': [0.27608541514724494]}
{'val_loss': [0.4046412229537964], 'loss': [0.3094890217622742]}
{'val_loss': [0.402709629945457], 'loss': [0.31274959692033005]}
{'val_loss': [0.6603823687881232], 'loss': [0.34584213881753384]}
{'val_loss': [0.4288881026208401], 'loss': [0.39256305201910435]}


In [0]:
### Descriptor loading and training for 10,0000 Triplets Trained on Clean using Tough Positive 
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=True)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=100000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=10000)

Using clean patches
100%|██████████| 116/116 [00:37<00:00,  3.10it/s]


100%|██████████| 10000/10000 [00:00<00:00, 68070.90it/s]


Using clean patches
100%|██████████| 116/116 [00:21<00:00,  3.13it/s]


100%|██████████| 1000/1000 [00:00<00:00, 68878.77it/s]


In [0]:
from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps

In [0]:
#Train on regular for 35 epochs
epochs = 35
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer
descriptor_history_full=[]
for e in range(epochs):
  
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  descriptor_history_full.append(descriptor_history)
  ### Saves optimizer and weights
  descriptor_model_trip.save('descriptor_farnegative.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@descriptor_farnegative.h5" https://file.io
  print("EPOCH NUMBER: {}".format(e))

Epoch 1/1
199/200 [============================>.] - ETA: 2s - loss: 0.4195

100%|██████████| 10000/10000 [00:00<00:00, 34572.50it/s]


200/200 [==============================] - 465s 2s/step - loss: 0.4214 - val_loss: 0.7304


100%|██████████| 1000/1000 [00:00<00:00, 68062.83it/s]


{"success":true,"key":"TR7T7A","link":"https://file.io/TR7T7A","expiry":"14 days"}EPOCH NUMBER: 0
Epoch 1/1
199/200 [============================>.] - ETA: 2s - loss: 0.5092

...

100%|██████████| 10000/10000 [00:00<00:00, 31108.51it/s]


200/200 [==============================] - 473s 2s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00


100%|██████████| 1000/1000 [00:00<00:00, 60773.80it/s]


{"success":true,"key":"vAJKoP","link":"https://file.io/vAJKoP","expiry":"14 days"}EPOCH NUMBER: 10
Epoch 1/1
160/200 [=======================>......] - ETA: 1:32 - loss: 0.0000e+00

KeyboardInterrupt: ignored

In [0]:
for epoch in descriptor_history_full:
  print(epoch.history)

In [0]:
from read_data_farnegative.py import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps

In [0]:
### Descriptor loading and training for 10,0000 Triplets Trained on Clean using Tough Positive 
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=True)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=100000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=10000)

In [0]:
#Train on farnegative for 5 epochs
epochs = 5
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer
descriptor_history_full=[]
for e in range(epochs):
  
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  descriptor_history_full.append(descriptor_history)
  ### Saves optimizer and weights
  descriptor_model_trip.save('descriptor_farnegative.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@descriptor_farnegative.h5" https://file.io
  print("EPOCH NUMBER: {}".format(e))

In [0]:
for epoch in descriptor_history_full:
  print(epoch.history)

In [0]:
denoise_model= keras.models.load_model('./DnCnn+TransferCnn.h5',custom_objects={'sum_squared_error':sum_squared_error})
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)

In [0]:
#Patch Verification:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

In [0]:
#Image Matching
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

In [0]:
#Patch Retrieval
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

### **Compressing and Saving:**

In [0]:
!zip -rq descriptors.zip ./out/custom

In [0]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
  file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
  }

  media = MediaFileUpload(path, 
                          mimetype='application/octet-stream',
                          resumable=True)

  created = drive_service.files().create(body=file_metadata,
                                  media_body=media,
                                  fields='id').execute()

  print('File ID: {}'.format(created.get('id')))

  return created

In [0]:
save_file_to_drive('descriptors_save.zip', 'descriptors.zip')